## Tensor by prrLU and CI

In [1]:
import numpy as np
from scipy.linalg import lu

def tensor_train_CI(T):
    """
    使用矩陣交叉插值（CI）將 3x3x3 張量 T 分解為 Tensor Train (MPS) 形式
    """
    # Step 1: Reshape T to a 3x9 matrix (將 i 視為行，(jk) 視為列)
    T_mat = T.reshape(3, 9)
    
    # Step 2: 進行 SVD 分解
    U, S, Vh = np.linalg.svd(T_mat, full_matrices=False)
    
    # Step 3: 取 rank-2 近似（保留前 2 個特徵值）
    rank = 2
    U_trunc = U[:, :rank]   # (3x2)
    S_trunc = np.diag(S[:rank])   # (2x2)
    V_trunc = Vh[:rank, :]  # (2x9)

    # Step 4: 重新排列 V，使其形狀為 (2, 3, 3)
    V_tensor = V_trunc.reshape(rank, 3, 3)

    # Step 5: 產生 Tensor Train
    M1 = U_trunc   # (3x2)
    M2 = np.tensordot(S_trunc, V_tensor, axes=(1, 0))  # (2x3x3)

    return M1, M2

def tensor_train_prrLU(T):
    """
    使用部分秩揭示 LU 分解（prrLU）將 3x3x3 張量 T 分解為 Tensor Train (MPS) 形式
    """
    # Step 1: Reshape T to a 3x9 matrix (i 為行, (jk) 為列)
    T_mat = T.reshape(3, 9)
    
    # Step 2: 執行 LU 分解
    P, L, U = lu(T_mat)

    # Step 3: 取 rank-2 近似
    rank = 2
    L_trunc = L[:, :rank]  # (3x2)
    U_trunc = U[:rank, :]  # (2x9)

    # Step 4: 重新排列 U 使其形狀為 (2, 3, 3)
    U_tensor = U_trunc.reshape(rank, 3, 3)

    # Step 5: 產生 Tensor Train
    M1 = L_trunc   # (3x2)
    M2 = U_tensor  # (2x3x3)

    return M1, M2

# 測試
T = np.random.rand(3, 3, 3)
M1_CI, M2_CI = tensor_train_CI(T)
M1_prrLU, M2_prrLU = tensor_train_prrLU(T)

# 打印結果
print("Tensor Train (CI):")
print("M1 shape:", M1_CI.shape)
print("M2 shape:", M2_CI.shape)

print("\nTensor Train (prrLU):")
print("M1 shape:", M1_prrLU.shape)
print("M2 shape:", M2_prrLU.shape)


Tensor Train (CI):
M1 shape: (3, 2)
M2 shape: (2, 3, 3)

Tensor Train (prrLU):
M1 shape: (3, 2)
M2 shape: (2, 3, 3)
